In [17]:
from data.repo import Repo
import pandas as pd
import utils.visuals as vs

Gettings a teams ids

In [11]:
id = Repo.static.get_team_id_by_abbr(abbr="GS")
id

9

### Example: Fetching Data From For a given team using its team id

In [13]:
team = Repo.teams.getTeamData(id=id)
x = [team.name, team.abbr, team.division]
y = ["Name", "Abbriviation", "Division"]
pd.DataFrame(x, y)

,0
Name,Warriors
Abbriviation,GS
Division,Pacific Division


### Summarizing a teams information

In [18]:
vs.Summarize(team)

,columns,data
0,Full Name,Golden State Warriors
1,Abbreviation,GS
2,Division,Pacific Division
3,Location,Golden State
4,Games Played,58
5,Win-Lose Record,29 - 29
6,Win Percentage,50.0
7,Points Scored,6880
8,Points Conceded,6875


How team are IDs collected?

In [21]:
import requests as req
import pandas as pd
import json
from IPython.core.display import JSON
from data.models.team_model import TeamModel
from data.repo import Repo

url = "http://site.api.espn.com/apis/site/v2/sports/basketball/nba/teams"
data = req.get(url).text

# Serialize the JSON and only extract the Sports Atrribute
data = json.loads(data)["sports"]
# Extract Leagues attribute
leagues = data[0]["leagues"]

# Extract the NBA
league = leagues[0]
# Get teams
teams = league["teams"]

ids = []
initials = []
display_names = []
    
# Extract every teams name, initial, short name and display name
for x in range(len(teams)):    
    ids.append(int(teams[x]["team"]["id"]))
    initials.append(teams[x]["team"]["abbreviation"])
    display_names.append(teams[x]["team"]["displayName"])

result = pd.DataFrame({
    'Id':ids,
    'Initial':initials,
    'Display Name':display_names
})

results = result.sort_values('Id')

results.to_csv()


',Id,Initial,Display Name\r\n0,1,ATL,Atlanta Hawks\r\n1,2,BOS,Boston Celtics\r\n18,3,NO,New Orleans Pelicans\r\n4,4,CHI,Chicago Bulls\r\n5,5,CLE,Cleveland Cavaliers\r\n6,6,DAL,Dallas Mavericks\r\n7,7,DEN,Denver Nuggets\r\n8,8,DET,Detroit Pistons\r\n9,9,GS,Golden State Warriors\r\n10,10,HOU,Houston Rockets\r\n11,11,IND,Indiana Pacers\r\n12,12,LAC,LA Clippers\r\n13,13,LAL,Los Angeles Lakers\r\n15,14,MIA,Miami Heat\r\n16,15,MIL,Milwaukee Bucks\r\n17,16,MIN,Minnesota Timberwolves\r\n2,17,BKN,Brooklyn Nets\r\n19,18,NY,New York Knicks\r\n21,19,ORL,Orlando Magic\r\n22,20,PHI,Philadelphia 76ers\r\n23,21,PHX,Phoenix Suns\r\n24,22,POR,Portland Trail Blazers\r\n25,23,SAC,Sacramento Kings\r\n26,24,SA,San Antonio Spurs\r\n20,25,OKC,Oklahoma City Thunder\r\n28,26,UTAH,Utah Jazz\r\n29,27,WSH,Washington Wizards\r\n27,28,TOR,Toronto Raptors\r\n14,29,MEM,Memphis Grizzlies\r\n3,30,CHA,Charlotte Hornets\r\n'

MySQL DB connection

In [25]:
import mysql.connector as mysql

db = mysql.connect(
    host="localhost",
    user="root",
    password="password",
)

db?

Type:           CMySQLConnection
String form:    <mysql.connector.connection_cext.CMySQLConnection object at 0x000001BEA4381310>
File:           c:\program files\python311\lib\site-packages\mysql\connector\connection_cext.py
Docstring:      Class initiating a MySQL Connection using Connector/C.
Init docstring: Initialization

Querying a table of team ids from database

In [28]:
import pandas as pd
db.connect()

cursor = db.cursor()

cursor.execute("Select * from nba.team_ids order by team_id;")
result = cursor.fetchall()
result = pd.DataFrame(result)
result


,0,1,2
0,1,ATL,Atlanta Hawks
1,2,BOS,Boston Celtics
2,3,BKN,Brooklyn Nets
3,4,CHA,Charlotte Hornets
4,5,CHI,Chicago Bulls
5,6,CLE,Cleveland Cavaliers
6,7,DAL,Dallas Mavericks
7,8,DEN,Denver Nuggets
8,9,DET,Detroit Pistons
9,10,GS,Golden State Warriors


A more cleaner table

In [29]:
sql = """select * from nba.team_ids
    order by team_id;
"""
cursor.execute(sql)
result = pd.DataFrame(cursor.fetchall())
ids = result[0]
abbr = result[1]
names = result[2]

result = pd.DataFrame({
    "Id":ids,
    "Abbriviation":abbr,
    "Name":names
})

result

,Id,Abbriviation,Name
0,1,ATL,Atlanta Hawks
1,2,BOS,Boston Celtics
2,3,BKN,Brooklyn Nets
3,4,CHA,Charlotte Hornets
4,5,CHI,Chicago Bulls
5,6,CLE,Cleveland Cavaliers
6,7,DAL,Dallas Mavericks
7,8,DEN,Denver Nuggets
8,9,DET,Detroit Pistons
9,10,GS,Golden State Warriors


 ### Using DATABASE CLASS

In [ ]:
from data.repo import Repo

team_ids = Repo.db.getTeamIDS()
team_ids

Quering for a single team

In [36]:
id = Repo.static.get_team_id_by_abbr(abbr="GS") + 1
cursor = db.cursor()
sql = f"""Select * from nba.team_ids 
                where team_id={id} 
              """

result = cursor.execute(sql)
result = cursor.fetchall()

result


[(10, 'GS', 'Golden State Warriors')]

Creating ShortTeamModel

In [ ]:
from data.models.short_team_model import ShortTeamModel

id = result[0][0]
abbr = result[0][1]
name = result[0][2]

model = ShortTeamModel(id, abbr, name)

Getting Team by ID from DB

In [ ]:
from data.repo import Repo
Repo.db.getTeamByID(id=12).name
